# Lesson 11: Model Toolbox

## Intro to Quantified Cognition

<a href="https://colab.research.google.com/github/compmem/QuantCog/blob/master/notebooks/11_Model_Toolbox.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Lesson plan

- Review some best practices
- Example of paramter recovery
- Discuss Modeler's Toolbox
- Example of Temporal Distinctiveness Model


## Best Practices

- Show model fit
- Assess parameter recovery
- Demonstrate selective influence
- Quantify uncertainty in parameter estimates
- Engage in model selection

(See great introduction by Heathcote et al., 2015)

## Parameter Recovery

- Use a model to generate data with known parameters
- Perform model fits to recover those parameters
- Poor parameter recovery suggests difficulty in model interpretation


## Selective Influence

- One option is demonstration of ecological validity 
  - (e.g., success in a real-world application)
- Barring that, we can perform experimental manipulations to isolate individual processes in the model
  - Speed-accuracy trade-off
  - Bias

## *ONLY* if on Google Colab

In [ ]:
# to install RunDEMC
!pip install git+https://github.com/compmem/RunDEMC.git


In [ ]:
# to retrieve the data
!wget https://raw.githubusercontent.com/compmem/QuantCog/master/notebooks/decision_data.csv

In [ ]:
# to retrieve the wfpt model
!wget https://raw.githubusercontent.com/compmem/QuantCog/master/notebooks/wfpt.py

## Load and process the data

In [ ]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import pandas as pd               # efficient tables
import matplotlib.pyplot as plt   # plotting
from scipy import stats

from RunDEMC.density import kdensity
from RunDEMC import Model, Param, dists, calc_bpic, joint_plot

from wfpt import wfpt_like, wfpt_gen

from joblib import Parallel, delayed
try:
    import scoop
    from scoop import futures
except ImportError:
    print("Error loading scoop, reverting to joblib.")
    scoop = None


In [ ]:
# load in the data
dat = pd.read_csv('decision_data.csv', index_col=0)
dat = dat[dat.cond != 'Neutral']
dat['rt_acc'] = dat['rt']
dat.loc[dat.correct==0,'rt_acc'] = -dat['rt']
dat.head()

In [ ]:
# set up the sim
nsims = 5000
cond='Accuracy'

# normed histogram
def dhist(dat, nvals, alpha=.3, color='b'):
    p,b = np.histogram(dat,bins='auto',density=True)
    w = b[1]-b[0]
    p *= float(len(dat))/nvals
    return plt.bar(b[1:]-w,p,width=w,alpha=.3,color=color)

# normed pdf
xvals = np.linspace(0.0,2.0,1000)
choices = np.concatenate([np.ones(len(xvals))*2, np.ones(len(xvals))*1])
rts = np.concatenate([xvals, xvals])

# put it all together
def run_wfpt(cond, v_mean, a, w_mode, w_std=0.0,
             v_std=0.0, t0=0.0, nsamp=5000, err=.0001):

    ndat = (dat['cond']==cond).sum()
    # plot the hist of the data, followed by the model PDF line
    dhist(np.array(dat[(dat['cond']==cond)&(dat['correct']==1)]['rt']), ndat, color='b')
    likes = wfpt_like(choices, rts, v_mean, a, w_mode, w_std=w_std,
                      v_std=v_std, t0=0, nsamp=nsamp, err=err)
    plt.plot(xvals+t0, likes[choices==2], color='b', lw=2.)
    
    dhist(np.array(dat[(dat['cond']==cond)&(dat['correct']==0)]['rt']), ndat, color='r')
    plt.plot(xvals+t0, likes[choices==1], color='r', lw=2.)
    #ylim(0,5.0)
    plt.xlim(0,2.0)

In [ ]:
# set up new figure
plt.figure(figsize=(10,6))

# try different params!
cond='Accuracy'

v_mean = 3.0
v_std = 2.0
a = 1.5
w_mode = 0.4
w_std = 0.1
t0 = .29

# call the function
run_wfpt(cond, v_mean, a, w_mode, w_std=w_std,
         v_std=v_std, t0=t0, nsamp=1000, err=.0001)


## Using computers to test hypotheses

In [ ]:
# grab the beh data of interest
choices_A = np.array(dat[(dat['cond']=='Accuracy')]['correct']+1)
rts_A = np.array(dat[(dat['cond']=='Accuracy')]['rt'])
choices_S = np.array(dat[(dat['cond']=='Speed')]['correct']+1)
rts_S = np.array(dat[(dat['cond']=='Speed')]['rt'])
print(len(choices_A), len(choices_S))

In [ ]:
# this is the required def for RunDEMC
def eval_fun(pop, *args):
    pnames = args[0]

    if scoop and scoop.IS_RUNNING:
        likes = list(futures.map(eval_mod, [indiv for indiv in pop],
                                 [pnames]*len(pop)))
    else:
        # use joblib
        likes = Parallel(n_jobs=-1)(delayed(eval_mod)(indiv, pnames)
                                    for indiv in pop)

    return np.array(likes)

In [ ]:
# Test change in threshold

# set up the params
params = [Param(name='v_A', prior=dists.normal(0., 2.)),
          #Param(name='v_S', prior=dists.normal(0., 2.)),
          Param(name='v_std', prior=dists.halfcauchy(2.0)),
          Param(name='w', prior=dists.normal(0, 1.4), transform=dists.invlogit),
          Param(name='w_std', prior=dists.halfcauchy(0.5)),
          Param(name='a_A', prior=dists.trunc_normal(2.0, 2.0, 0., 5.0)),
          #Param(name='a_S', prior=dists.trunc_normal(2.0, 2.0, 0., 5.0)),
          Param(name='t0', prior=dists.trunc_normal(.2, 1.0, 0., 1.0))]
param_names = [p.name for p in params]

# define the likelihood function
def eval_mod(params, param_names):
    log_like = 0.0
    p = {param_names[j]: params[j] for j in range(len(params))}
    
    # first Accuracy
    likes_A = wfpt_like(choices_A, rts_A, 
                        v_mean=p['v_A'], v_std=p['v_std'], a=p['a_A'], 
                        w_mode=p['w'], t0=p['t0'], w_std=p['w_std'], 
                        nsamp=1000)
    log_like += np.log(likes_A).sum()

    # then Speed
    #likes_S = wfpt_like(choices_S, rts_S, 
    #                    v_mean=p['v_S'], v_std=p['v_std'], a=p['a_S'], 
    #                    w_mode=p['w'], t0=p['t0'], w_std=p['w_std'], 
    #                    nsamp=1000)
    #log_like += np.log(likes_S).sum()

    return log_like
        
# make the model
m = Model('all', params=params,
          like_fun=eval_fun,
          like_args=(param_names,),
          #purify_every=5,
          verbose=True)


In [ ]:
# do some burnin
times = m.sample(75, burnin=True)

In [ ]:
plt.plot(m.weights[30:]);

In [ ]:
plt.plot(m.particles[10:, :, 3]);

In [ ]:
print("Best fitting params:")
burnin=5
best_ind = m.weights[burnin:].argmax()
print("Weight:", m.weights[burnin:].ravel()[best_ind])
indiv = [m.particles[burnin:,:,i].ravel()[best_ind] 
         for i in range(m.particles.shape[-1])]
pp = {}
for p,v in zip(m.param_names,indiv):
    pp[p] = v
    print('"%s": %f,'%(p,v))

## LBA fit to Accuracy condition

In [ ]:
# grab the beh data of interest
dat_A_1 = np.array(dat[(dat['cond']=='Accuracy')&(dat['correct']==0)]['rt'])
dat_A_2 = np.array(dat[(dat['cond']=='Accuracy')&(dat['correct']==1)]['rt'])
num_A = float((dat['cond']=='Accuracy').sum())
prop_A_1 = len(dat_A_1)/num_A
prop_A_2 = len(dat_A_2)/num_A
dat_S_1 = np.array(dat[(dat['cond']=='Speed')&(dat['correct']==0)]['rt'])
dat_S_2 = np.array(dat[(dat['cond']=='Speed')&(dat['correct']==1)]['rt'])
num_S = float((dat['cond']=='Speed').sum())
prop_S_1 = len(dat_S_1)/num_S
prop_S_2 = len(dat_S_2)/num_S


In [ ]:
def lba_sim(I=(1.0,1.5), A=.1, S=1.0, b=1.0, t0=0.0, 
            num_sims=1000, max_time=2., I_scales_S=False, **kwargs):
    # set drift rate from inputs
    dr = np.float64(I)
    
    # set the number of choices
    nc = len(dr)
    
    # pick starting points
    k = np.random.uniform(0., A, (num_sims, nc))
    
    # pick drifts
    if I_scales_S:
        # calc S from drift rates
        S = np.sqrt((dr**2).sum())*S
        
    # must make sure at least one d is greater than zero for each sim
    d = np.random.normal(dr, S, (num_sims, nc))
    
    # see where there are none above zero
    #ind = np.all(d<=0.0,axis=1)
    #while np.any(ind):
    #    d[ind,:] = np.random.normal(dr,S,(ind.sum(),nc))
    #    ind = np.all(d<=0.0,axis=1)

    # clip it to avoid divide by zeros
    d[d<=0.0] = np.finfo(dr.dtype).eps

    # calc the times for each
    t = (b-k)/d

    # see the earliest for each resp
    inds = t.argmin(1)
    times = t.take(inds+np.arange(t.shape[0])*t.shape[1])

    # process into choices
    times += t0
    
    # get valid responses
    resp_ind = times < (max_time)
    resp = inds+1
    resp[~resp_ind] = 0
    
    # return as data frame
    return pd.DataFrame.from_dict({'response':resp, 'rt':times})
    


In [ ]:
# *** INCOMPLETE ***

# set up the params
params = [Param(name='d1', prior=dists.trunc_normal(1., 2., 0., 10.)),
          Param(name='d2', prior=dists.trunc_normal(1., 2., 0., 10.)),
          Param(name='A', prior=dists.trunc_normal(.5, 2.0, 0., 2.0)),
          Param(name='b_A', prior=dists.trunc_normal(2.0, 2.0, 0., 5.0)),
          Param(name='b_S', prior=dists.trunc_normal(2.0, 2.0, 0., 5.0)),
          Param(name='t0', prior=dists.trunc_normal(.2, 1.0, 0., 1.0))]
param_names = [p.name for p in params]

# define the likelihood function
nsims = 10000
max_time = 2.0
extrema = (0, max_time)
S = 1.0
def eval_mod(params, param_names):
    log_like = 0.0
    p = {param_names[j]: params[j] for j in range(len(params))}
    

    # first check for simple issues
    if (p['A'] > p['b_A']) or (p['A'] > p['b_S']) or np.any(params<0):
        # A can't be bigger than b
        likes[i] = -np.inf
        continue

    # first Accuracy
    res = lba_sim(I=(p['d1'], p['d2']), A=p['A'], S=S, b=p['b_A'], 
                  num_sims=nsims, max_time=max_time, t0=p['t0'])

    # process first response
    rts = res.loc[res['response'] == 1, 'rt']
    if len(rts)>2:
        pp,xx = kdensity(rts, extrema=extrema, xx=dat_A_1)
        pp *= float(len(rts))/nsims
        likes[i] += np.log(pp).sum()
    else:
        likes[i] = -np.inf
        continue

    # process second response
    rts = res.loc[res['response'] == 2, 'rt']
    if len(rts)>2:
        pp,xx = kdensity(rts, extrema=extrema, xx=dat_A_2)
        pp *= float(len(rts))/nsims
        likes[i] += np.log(pp).sum()
    else:
        likes[i] = -np.inf
        continue

    # then Speed
    res = lba_sim(I=(p['d1'], p['d2']), A=p['A'], S=S, b=p['b_S'], 
                  num_sims=nsims, max_time=max_time, t0=p['t0'])

    # process first response
    rts = res.loc[res['response'] == 1, 'rt']
    if len(rts)>2:
        pp,xx = kdensity(rts, extrema=extrema, xx=dat_S_1)
        pp *= float(len(rts))/nsims
        likes[i] += np.log(pp).sum()
    else:
        likes[i] = -np.inf
        continue

    # process second response
    rts = res.loc[res['response'] == 2, 'rt']
    if len(rts)>2:
        pp,xx = kdensity(rts, extrema=extrema, xx=dat_S_2)
        pp *= float(len(rts))/nsims
        likes[i] += np.log(pp).sum()
    else:
        likes[i] = -np.inf
        continue

    return likes
        
# make the model
m_t = Model('thresh', params=params,
            like_fun=like_fun,
            like_args=(dat,),
            purify_every=5,
            verbose=True)


## Model Toolkit

- What are the coding equivalents for common cognitive processes?

## Items/Stimuli

- A cognitive model must *represent* the stimuli in your experiment. 
- This can be as simple as a vector


In [ ]:
# orthogonal items in a list-learning experiment
nitems = 10
items = np.eye(nitems)

plt.imshow(items)

## Overlapping items

- Sometimes it's necessary for items to overlap (i.e., perceptually or semantically)
- Here you can simply add columns to your items

In [ ]:
# overlapping items in a list-learning experiment
nitems = 10
ncolors = 2
nfeatures = nitems + ncolors
items = np.eye(nfeatures)
items = items[:nitems]

# add in color features
items[1::2, -1] = 1
items[::2, -2] = 1

plt.imshow(items)

## Working memory

- We can keep stimuli active, even when not just-presented
- Some options are:
  - buffers (e.g., SAM, Atkinson & Shiffrin)
  - exponential decay (e.g., TCM, Howard & Kahana)
  - time cells (e.g., SITH/TILT, Howard & Shankar)

In [ ]:
# present a list of items and store in working memory (context)
c = np.zeros(nfeatures)
c_save = []
rho = .5
for i in items:
    c = rho*c + (1-rho)*i
    c_save.append(c)
plt.imshow(c_save)

## Associations

- In order to have a longer-term memory, we must form associations
- If items (and other information, such as working memory) is a vector, you can store and update associations in a matrix
- There are *many* potential associative memory rules:
  - Hebbian (simple outer product)
  - Prediction-error (e.g., Rescorla--Wagner)

In [ ]:
# form an outer product between an item and context
M = np.zeros((nfeatures, nfeatures))
for i in range(len(items)):
    M = M + np.outer(items[i], c_save[i])
plt.imshow(M)

## Querying Memory

- Once we have information stored in a matrix, you can pull it out with matrix multiplication

In [ ]:
s = np.dot(M, c_save[4])
plt.plot(s)

## Decisions

- If we have a strength value, we still need to generate a behavior
- Here we can employ many types of decision rules
- If we don't care about reaction-times, we can use a softmax rule:

$$\frac{e^{\tau s_i}}{\sum_{j} e^{\tau s_j}}$$

## Temporal Distinctiveness Example

In Siefke et al. (2019, *Memory & Cognition*) we wanted to test what it meant for an experience to be disctinctive:

https://link.springer.com/content/pdf/10.3758%2Fs13421-019-00925-5.pdf